In [1]:
# Lab settings - please ingnore
# Make plots a resonable size
options(repr.plot.width=7, repr.plot.height=3.5)

<div class="big_title">LAB 5: Advanced data manipulation</div>

BIO3782: Biologist's Toolkit (Dalhousie University)

----------------------------------------

# Setup of workspace

<span class="important"></span> Make sure all required files are in the working directory:

* Create a **folder** on the "Desktop" and name it <span class='file'>Lab5</span>
* From Brightspace, download the following files into your new folder <span class='file'>Desktop\Lab5</span>:
   * <span class='file'>surveys_clean.csv</span>
   
* In RStudio, change the "working directory" to: <span class='file'>Desktop\Lab5</span>. Click here if you need a [refresher on the **working directory**](https://diego-ibarra.github.io/biol3782/week1/index.html#RStudio's-%22Working-Directory%22)

----------------------------------


# Data manipulation

The path between data collection and the final plot or analysis is a publication is, to say the least, complicated and convoluted. Raw data files contain data organized in a way that preserves all the information that was available during collection, often in a format that facilitates data entry. For each different research objective — whether it is to test a hypothesis, do a plot or run an analysis — the raw data often needs to be filtered, splitted, joined or merged with other data, reshaped, mutated, summarized, and/or many other processes and trasformations. All of these intermediate steps, which are necessary to get the raw data into a shape or format suitable for the final analysis, are collective know as **Data manipulation**.


# The tidyverse

The `tidyverse` package is an "umbrella-package" (or "bundle-package") that installs `tidyr`, `dplyr`, and several other packages useful for data analysis, such as `ggplot2`, `tibble`, etc. 

The tidyverse package tries to address 3 common issues that arise when doing data analysis with some of the built-in functions that come with R:

1. The results from a base R function sometimes depend on the type of data.

2. Using R expressions in a non standard way, which can be confusing for new learners.

3. Hidden arguments, having default operations that new learners are not aware of.

Here we will only install and load 
    
You should already have installed and loaded the `tidyverse` package. If we haven't already done so, we can type install.packages("tidyverse") straight into the console. Then, to load the package type library(tidyverse)

Let's download the [dplyr cheat sheet] (https://github.com/rstudio/cheatsheets/raw/master/data-transformation.pdf) for reference as well 

In [ ]:
install.packages("tidyverse")

In [ ]:
library(tidyverse)

# What are `dplyr` and `tidyr`?


`dplyr` is a package for making tabular data manipulation easier. It pairs nicely with tidyr which enables you to swiftly convert between different data formats for plotting and analysis.

The package `dplyr` provides easy tools for the most common data manipulation tasks. It is built to work directly with data frames, with many common tasks optimized by being written in a compiled language (C++). An additional feature is the ability to work directly with data stored in an external database. The benefits of doing this are that the data can be managed natively in a relational database, queries can be conducted on that database, and only the results of the query are returned.

This addresses a common problem with R in that all operations are conducted in-memory and thus the amount of data you can work with is limited by available memory. The database connections essentially remove that limitation in that you can connect to a database of many hundreds of GB, conduct queries on it directly, and pull back into R only what you need for analysis.

The package `tidyr` addresses the common problem of wanting to reshape your data for plotting and use by different R functions. Sometimes we want data sets where we have one row per measurement. Sometimes we want a data frame where each measurement type has its own column, and rows are instead more aggregated groups (e.g., a time period, an experimental unit like a plot or a batch number). Moving back and forth between these formats is non-trivial, and `tidyr` gives you tools for this and more sophisticated data manipulation.

## The experiment 
We conducted an experiment to evaluate the cumulative effects of drought and disease on the silver tree. We measured physiological responses of the plant such as photosynthetic oxygen evolution and electrical conductivity.

First, we'll download the data. 


As before, we'll read in our data using the `read_csv()` function from the tidyverse package `readr`. We will need to do the following:

1. Make a new directory on your Desktop and call it Lab5
1. Download the <span class="file">Silver Tree Study photosynthetis.csv</span> and <span class="file">Silver tree study conductance.csv</span> datasets from Brightspace and place them in your Lab5 directory.
1. Open R Studio and change your working directory to Lab5. If you don't remember how to do this, review [this section](https://diego-ibarra.github.io/biol3782/week1/index.html#RStudio's-%22Working-Directory%22)

In [ ]:
photosynthesis <- read_csv("Silver Tree Study photosynthetis.csv")

conductivity <- read_csv("Silver tree study conductance.csv")

Let's take a look at the two datasets below:

In [ ]:
head(photosynthesis) # head just displays the first 6 rows

In [ ]:
str(photosynthesis)

Let's take a look at Photosynthesis as a function of Species graphically. 

In [ ]:
ggplot(data=photosynthesis,
       aes(x=factor(Species), y=Photosynthesis, fill=factor(Species)))+
geom_boxplot()+
scale_y_continuous(limits=c(-10,50))+ #sets axis limits for y-axis. We note that this will remove values > 50. A warning message will let us know how many observations were excluded (10 observations)
theme_classic()+
guides(fill=FALSE) #removes legend for fill

<div class="Q"><br><br>

how many rows will the function `head` display? 

ANSWER: 6<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

What is the function to display the last 6 rows of a dataframe (opposite of `head`)? 

ANSWER: tail()<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

The function `str()` shows the structure of the dataframe? 

ANSWER: TRUE<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How many columns/variables are there in `photosynthesis`? 

ANSWER: 12<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

What class of variable is `Species`?

ANSWER: Character
    
    other multiple choice options:
    Double/numeric
    Factor


    
<br><br><div class="qhelp"></div></div>

In [ ]:
head(conductivity)

In [ ]:
str(conductivity)

In [ ]:
ggplot(data=conductivity,
       aes(x=factor(Species), y=Conductance, fill=factor(Species)))+
geom_boxplot()+
theme_classic()+
guides(fill=FALSE) #removes legend for fill

<div class="Q"><br><br>

How many observations are in `conductivity`? 

ANSWER: 1722<br><br>

<br><br><div class="qhelp"></div></div>

Since the experiment was run with two different instruments, we ended up with two different datasets of paired measurements. We can combined both datasets into one more useful dataset.

# Combining two or more datasets

There are several functions that allow you to combined multiple datasets together to be able to access data from all of them.

## Join

`Join` functions allow you to connect multiple tables based on common variables(columns).

### left_join

The most useful of these is a `left join`, which takes a table (say table 1), and then adds all matching records from table 2.The resulting table will show all records from table 1 (the left hand table), and matching records from table 2 (the right hand table).

The basic syntax of `left_join` is listed below

`left_join`(dataframe 1, dateframe 2, by = "variable names")

R will automatically detect which columns two tables have in common IF they are called the same thing. If not, you have to feed the matching columns names into the join function. 

<img src="left_join.png" width="500px">

We can create a new dataframe with information all the information from both `photosynthesis` and `conductivity` joining the two dataframes using the qualifying variables in the columns from "Sample type"to "Obs".


In [ ]:
combined_data <- left_join(photosynthesis,conductivity, 
                           by =c("Sample type",
                                 "Days after inoculation", 
                                 "Date", 
                                 "Licor", 
                                 "Trial",
                                 "Treatment",
                                "Species",
                                "Plant Number",
                                "Isolate Number", 
                                "Unique Sample Number",
                                "Obs"))

Let's look at the combined dataset below

In [ ]:
head(combined_data)

In [ ]:
str(combined_data)

We can see that `left_join` combined `photosynthesis` and `conductivity` data by all the criteria columns we specified above. We basically get `photosynthesis` back, but with the addition of variables Conductance and Ci, which is unique to `conductivity`. 

<div class="Q"><br><br>

How many observations(rows) does `combined_data` have? 

ANSWER: 1722<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How many columns does `combined_data` have? 

ANSWER: 14<br><br>

<br><br><div class="qhelp"></div></div>

### inner_join

`inner_join` returns all rows from dataframe 1 where there are matching values in dataframe 2, and all columns from dataframe 1  and dataframe 2. If there are multiple matches between them, all combination of the matches are returned. Basically, the result shows only 'matched' records - where there is a matching value in both tables.

<img src="inner_join.png" width="500px">

In [ ]:
combined_inner<-inner_join(photosynthesis, conductivity, 
                           by =c("Sample type",
                                 "Days after inoculation",
                                 "Date", 
                                 "Trial",
                                 "Treatment",
                                "Species",
                                "Plant Number",
                                "Isolate Number", 
                                "Unique Sample Number",
                                "Obs"))

head(combined_inner)

Notice that we get an repeated variables `Licor.x` and `Licor.y`. We didn't specify that `Licor' was a criteria for joining and since it appears in both datasets, join will create new variables for them; once from each dataset. 

<div class="Q"><br><br>
What variables would be duplicated if we ran the following code?
    
inner_join(photosynthesis, conductivity, 
                           by =c("Sample type",
                                 "Days after inoculation",
                                 "Date", 
                                 "Trial",
                                 "Treatment",
                                "Species",
                                "Plant Number",
                                "Isolate Number", 
                                "Unique Sample Number"
                                ))

ANSWER: Licor and Obs<br><br>

<br><br><div class="qhelp"></div></div>

### full_join

`full_join` will return all rows and all columns from both dataframe 1 and dataframe 2. It will return NA for instances where there are no matching values.

<img src="full_join.png" width="500px">

In [ ]:
combined_full<-full_join(photosynthesis, conductivity)

head(combined_full)

In [ ]:
str(combined_full)

Notice that this returns pretty much the same dataset as the one we created with `left_join`. We didn't have to specify any of the variables to join by in this case. We get all varaibles and rows from both `photosynthesis` and `conductivity`.

<div class="Q"><br><br>

How many observations/rows does the variable Ci have in the `combined_full` dataset? 

ANSWER: 1722<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

Are the variable classes of `combined_full` and `combined_data` the same? 

ANSWER: YES<br><br>

<br><br><div class="qhelp"></div></div>

### anti_join

`anti_join` removes all records from dataframe 1 where that have matching variables in dataframe 2. This basically filters your datasets.

<img src="anti_join.png" width="500px">

In [ ]:
combined_anti<-anti_join(photosynthesis, conductivity)

head(combined_anti)

In [ ]:
str(combined_anti)

Notice that we effectively returned all variables of `photosynthesis` without any data! This happened because `conductivity` had the all same variables and these measurements were paired with those of ohotosynthesis (taken from the same sample at the same time). There were no instances where the variables didn't match that the `anti_join` function could return and observation. 

<div class="Q"><br><br>

How many variables does `combined_anti` have? 

ANSWER: 12<br><br>

<br><br><div class="qhelp"></div></div>

## Merge

Another way to combined datasets is to use the function `merge()`. By default the data frames are merged on the columns with names they both have, but separate specifications of the columns can be given by by.x and by.y. The rows in the two data frames that match on the specified columns are extracted, and joined together. If there is more than one match, all possible matches contribute one row each. Thie basically merges two datasets horizontally.

<img src="merge.png" width="500px">

<br><br><br>
<div class="startTASK"></div>

Using the help section, look at the syntax for merge

<div class="endTASK"></div>
<br><br><br>

Let's recreate the `combined_data` dataset using merge instead.

In [ ]:
combined_data<-merge(photosynthesis,conductivity,all.x = TRUE, all.y = TRUE)

head(combined_data) 

In [ ]:
str(combined_data)

<div class="Q"><br><br>

What does the arguement "by" in the merge function do? 

ANSWER: specifies the columns used as classifiers for merging<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How many variables does `combined_data` have? 

ANSWER: 14<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How extra variable does `combined_data` have that `conductivity` does not? 

ANSWER: Photosynthesis<br><br>
    
    other options for multiple choice:
    Species
    Trial
    Obs

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

Are the variable classes of `combined_data` created using `left_join` and `merge` the same? 

ANSWER: YES<br><br>

<br><br><div class="qhelp"></div></div>

<br><br><br>
<div class="startTASK"></div>

Let us download the task datasets <span class="file">task_data_fox_size.csv</span> and <span class="file">task_data_fox_weight.csv</span> 

Using `Joins` or `merge`, create a new dataset called `fox_data` combining data from both <span class="file">task_data_fox_size.csv</span> and <span class="file">task_data_fox_weight.csv</span> 

<div class="endTASK"></div>

<br><br><br>

<div class="Q"><br><br>

How many rows does fox_data have? 

ANSWER: 30
    
    other options: 11911
    6
    31
    <br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How many columns does fox_data have? 

ANSWER: 6<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

Using the `summary` function, what is the median value of avfood? 

ANSWER: 0.6994<br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

What class of variable is ID? 

ANSWER: numeric<br><br>

<br><br><div class="qhelp"></div></div>

Next, we're going to learn some of the most common `dplyr` functions:

* `select()`: subset columns
* `filter()`: subset rows on conditions
* `mutate()`: create new columns by using information from other columns
* `group_by()` and `summarize()`: create summary statistics on grouped data
* `arrange()`: sort results
* `count()`: count discrete values

_________________________________________________________________________________________________________________________

# Selecting columns

If we only want to keep certain columns of data, we can use `select()`. The first argument to this function is the dataset, and the subsequent arguments are the columns to keep.

select(data, column names)

<img src="select.png" width="500px">

In the following code, we have selected the columns "Sample type", "Photosynthesis", and "Conductance". 

In [ ]:
sml_data<-select(combined_data, `Sample type`,Photosynthesis, Conductance)

head(sml_data)

In [ ]:
str(sml_data)

We can also use `:` to select multiple columns from left to right.

In [ ]:
sml_data_select<-select(combined_data, `Sample type`:Species,Photosynthesis, Conductance)

head(sml_data_select)

In [ ]:
str(sml_data_select)

To select all columns except certain ones, put a "-" in front of the variable to exclude it. In the following code, we want to exclude the columns "Licor" and "Obs"

In [ ]:
sml_data_omit<-select(combined_data, -Licor, -Obs)

head(sml_data_omit)

In [ ]:
str(sml_data_omit)

<div class="Q"><br><br>

Using the "combined_data" dataset and the "-", what would the code look like to select all columns except Species?
    
ANSWER to be removed in final iteration: select(combined_data, -Species) <br><br>
    
    
    other mutiple choice options: 
    
    select(combined_data, `Sample type`:Treatment, `Plant Number`:Conductance)
    
    select(combined_data, Species)
    
    select(combined_data, -Species) 

<br><br><div class="qhelp"></div></div>

# Filtering rows

What if we only want to select the data from one Species? You can also choose rows based on a specific criterion using the `filter()` function. The syntax for `filter()` is found below

filter(dataset, selection criteria)

<img src="filter.png" width="500px">

In the following code, we want to select all data from Indigenous Pathogen Species only.

In [ ]:
head(filter(combined_data, Species == "Indigenous Pathogen")) 

We can also filter by multiple criteria. Say we want Obs 1 data from Indigenous Pathogen Species only. We can use the `filter()` function and expand out selection criteria.

In [ ]:
head(filter(combined_data, Species == "Indigenous Pathogen", Obs ==1))

Let's take a look at our Species data to see if we can clean up our dataset. We note that Species is a character column. Using the function `as.factor()` we can convert Species entries into factors then use the `levels()` function to examine the factor levels.

In [ ]:
#convert to factor

combined_data$Species<-as.factor(combined_data$Species)

# provides levels of a factor using the levels command.  

levels(combined_data$Species)  

For simplicity, let's create a new object `data_filtered` and select only rows where Photosyntheis < 200 and Species is not equal to "Both Pathogens". Let's take a look at the first 4 rows.

In [ ]:
data_filtered<- filter(combined_data,Photosynthesis <200 & Species !="Both Pathogens")

head(data_filtered)

In [ ]:
str(data_filtered)

In [ ]:
summary(data_filtered$Species)

Note that "Both Pathogens" still appears as a level of `Species` in `data_filtered` but all the observations have been removed.
Let's compare Photosynthesis vs Conductance between `combined_data` and `data_filtered` graphically.

In [ ]:
ggplot(data=combined_data,
      aes(x=Photosynthesis, y=Conductance, fill=Species))+
geom_point(shape=21)+
theme_classic()+
labs(title="combined_data")

In [ ]:
ggplot(data=data_filtered,
      aes(x=Photosynthesis, y=Conductance, fill=Species))+
geom_point(shape=21)+
theme_classic()+
labs(title="data_filtered")

 In `data_filtered`, outlying data points (Photosynthesis>200) have been filtered out making it easier to visualize our data.

<div class="Q"><br><br>

What does `!=` in the `select()` funtion mean?
    
ANSWER to be removed in final iteration: Not equal to <br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

How many levels does Species have in `data_filtered`?
    
ANSWER :4 <br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

TRUE or FALSE: "Both Pathogens" in `data_filtered` has 0 observations because they did not take measurements?
    
ANSWER :FALSE <br><br>

<br><br><div class="qhelp"></div></div>

<br><br><br>
<div class="startTASK"></div>

Create a dataset called `male_fox_data` wherein only data from male foxes are included.

<div class="endTASK"></div>
<br><br><br>

<div class="Q"><br><br>

How many rows does `male_fox_data` have?
    
ANSWER to be removed in final iteration: 15 <br><br>

<br><br><div class="qhelp"></div></div>


# Nested functions

What if you want to select and filter at the same time? There are three ways to do this: use intermediate steps, nested functions, or pipes.

What if we wanted to select only Sample type, Photosynthesis and Conductance from the Control and Indigenous Pathogen Species? We could create multiple objects.

We can use `%in%` with `select()`to specify a list of paramters/levels we want to keep.

In [ ]:
#Creates a new data frame with rows containing only data from Indigenous Pathogen and Control Species
species_filtered <- filter(data_filtered, Species %in% c("Indigenous Pathogen", "Control")) 

#Then only selects the columns "Sample type", Species,Photosynthesis, Conductance)  
 species_sml<- select(species_filtered, `Sample type`, Species, Photosynthesis, Conductance)

#inspect new dataframe

head(species_sml) 

In [ ]:
str(species_sml)

This is readable, but is clunky and can clutter up your workspace with lots of small objects that youd have to keep track of.

Another method would be to nest functions (i.e. one function inside of another), like this:

<img src="nested.png" width="500px">

In [ ]:
species_sml <- select(filter(data_filtered, Species %in% c("Indigenous Pathogen", "Control")),
                      `Sample type`, Species, Photosynthesis, Conductance)

head(species_sml)

In [ ]:
str(species_sml)

R evaluates the expression from the inside out (in this case, filtering, then selecting). **I would avoid this method since it makes debugging your code a nightmare!**

# Pipes

A better method is to use pipes. Pipes let you take the output of one function and send it directly to the next, which is useful when you need to do many things to the same dataset. Pipes in R look like `%>%` and are made available via the `magrittr` package, installed automatically with `dplyr`. 

If you use RStudio, you can type the pipe with `Ctrl + Shift + M` if you have a PC or `Cmd + Shift + M` if you have a Mac.

We will re-create the `species_sml` dataset using pipes. First, we want to filter data from the `data_filtered` dataset, keeping rows where Species is either "Indigenous Pathogen" or "control", then select only the Sample type, Species, Photosynthesis, Conductance columns. We will use the pipe to send the `data_filtered` dataset through the `filter()` then `select()` functions. 

In [ ]:
species_sml<-data_filtered %>%
  filter(Species %in% c("Indigenous Pathogen", "Control")) %>% 
  select(`Sample type`, Species, Photosynthesis, Conductance)  

species_sml %>% 
head(4)

In [ ]:
str(species_sml)

In [ ]:
species_sml %>% 
ggplot()+
geom_boxplot(aes(x=Species, y=Photosynthesis, fill=Species))+
theme_classic()

Since `%>%` takes the object on its left and passes it as the first argument to the function on its right, we don't need to explicitly include the data frame as an argument to the `filter()` and `select()` functions any more.

<img src="pipes.png" width="500px">

<br><br><br>
<div class="startTASK"></div>

Using the `fox_data` dataset, create a new object `fox_data_cleaned` that contains only the columns ID, Sex, avfood, from female foxes only.
<div class="endTASK"></div>
<br><br><br>

<div class="Q"><br><br>

How many rows are there in `fox_data_cleaned`?
    
ANSWER to be removed in final iteration: 15 <br><br>
    
<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

What is the median weight of female foxes in the `fox_data_cleaned` dataset?
    
ANSWER:4.399 <br><br>
    
<br><br><div class="qhelp"></div></div>

_________________________________________________________________________

# Reshaping your dataframe with **pivot_longer** and **pivot_wider()**

Useful links:

1. [Pivot tutorial](https://tidyr.tidyverse.org/articles/pivot.html)
1. [pivot_longer](https://rdrr.io/github/tidyverse/tidyr/man/pivot_longer.html)
1. [pivot_wider](https://tidyr.tidyverse.org/reference/pivot_wider.html)

Sometimes you may need to alter the layout of tabluar datasets while preserving the values and relationships contained within. This can also be very useful when plotting data from very large datasets with multiple columns. You may have heard of `spread()` and `gather()`. These are functions with similar but more limited functionality and were replaced by `pivot_wider()` and `pivot_longer()`.

## pivot_wider

**pivot_wider** moves rows into columns (long to wide format). Values in the "Parameter" column become new column names and their respective values get copied into the reshaped data. Columns with no observations will get coded with a value of NA.

<img src="pivot_wider.png" width="500px">

Let's use `pivot_wider()` to turn Species factors into column names by changing the dataset from long to wide format.

In [ ]:
data_wide<-data_filtered  %>% 
filter(Treatment=="Drought", Obs==1) %>% 
select(Species, `Sample type`, Date, Photosynthesis) %>% 
pivot_wider(names_from="Species", values_from="Photosynthesis") 

data_wide %>% 
head(4)

In [ ]:
str(data_wide)

Notice that some columns have NA values. `pivot_wider` creates that value if it doesn't exist in the data.

Let's try to plot the data from the Control, Exotic Pathogen, and Indigenous Pathogen columns by date. In the wide format we will have to create 3 separate plots and used a plotting package like `cowplot` to acheive what we want. Wide format is nice for people that are not used to seeing data separated into value (i.e. mg/L O2) and classifier/parameter (i.e. ogygen) columns (like what you'll see in a little bit).

In [ ]:
control<-data_wide %>% 
ggplot()+
geom_boxplot(aes(x=Date, y= Control))+
theme_classic()+
theme(axis.text.x=element_text(angle=90)) #roates X-axis text to 90 degrees

exotic<-data_wide %>% 
ggplot()+
geom_boxplot(aes(x=Date, y= `Exotic Pathogen`))+
theme_classic()+
theme(axis.text.x=element_text(angle=90))

indigenous<-data_wide %>% 
ggplot()+
geom_boxplot(aes(x=Date, y= `Indigenous Pathogen`))+
theme_classic()+
theme(axis.text.x=element_text(angle=90))

cowplot::plot_grid(control, exotic,indigenous, nrow=1)

<div class="Q"><br><br>

How many observations does `data_wide` have?
    
ANSWER:73 <br><br>

<br><br><div class="qhelp"></div></div>

## pivot_longer

In this situation we are gathering the column names and turning them into a pair of new variables. One variable represents the column names as values, and the other variable contains the values previously associated with the column names.

**pivot_longer** moves columns into rows (wide to long format). `pivot_longer` takes several arguements but the most important ones for this exercise are `names_to` and `values_to`. `names_to` creates a new classifier/criteria column, taking arguements from your column names. `values_to` takes values from those columns and creates a new column of values classfied by the factors created by `names_to`.

<img src="pivot_longer.png" width="500px">

Let's use `pivot_longer` to transform `data_wide` back into `data_filtered`.

In [ ]:
data_long<-data_wide %>%
pivot_longer(names_to="Species", values_to="Photosynthesis", values_drop_na = TRUE, !`Sample type`:Date) 

data_long%>% 
head(4)

In [ ]:
str(data_long)

We have bassically created a column for values called Photosynthesis and a classifier/parameter column called Species. Let's try and plot Photosynthesis for all three species by date again.

In [ ]:
ggplot(data_long,
       aes(x=Date, y=Photosynthesis, fill=factor(Species)))+
       geom_boxplot()+
theme_classic()+
theme(axis.text.x=element_text(angle=90))

We got a similar graphical result with less lines of code! Long format is great when you need to plot data with multiple factor levels. 

<div class="Q"><br><br>

How many rows does `data_long` have?
    
ANSWER:73 <br><br>

<br><br><div class="qhelp"></div></div>

<div class="Q"><br><br>

TRUE or FALSE: `data_wide` and `data_long` have the same number ofnobservations because `pivot_wider` fills missing observations with NA?
    
ANSWER:TRUE <br><br>

<br><br><div class="qhelp"></div></div>

_______________________________________________________________________________________________

# Mutate

We can use `mutate()` to create new columns based on the values in existing columns, for example to do unit conversions, or to find the ratio of values in two columns.

For example, we can convert Photosynthesis oxygen measurements from mg/L to ug/L by creating a new column called `Photosynthesis_ppb`

<img src="mutate.png" width="500px">

In [ ]:
unit_conversion<-data_filtered %>%
  mutate(Photosynthesis_ppb = Photosynthesis*1000) %>% 
select(`Sample type`, Photosynthesis, Photosynthesis_ppb) 

unit_conversion%>% 
head(4)

In [ ]:
str(unit_conversion)

In [ ]:
unit_conversion %>% 
ggplot(aes(x=Photosynthesis, y=Photosynthesis_ppb))+
geom_point()+
theme_classic()

In [ ]:
summary(unit_conversion)

Notice that summary statistics after the mutation are 1000 times the original.

In [ ]:
We can also use mutate to change variable level names.

In [ ]:
change_lables<-data_filtered %>%
mutate(Species=factor(Species,
                     levels=c("Control", "Exotic Pathogen","Indigenous Pathogen"),
                     labels =c("Control", "P.Cinnamomi", "P.Multivora")))

levels(change_lables$Species)

We can also use `mutate()` to change variable types. Here, we will convert some variables from character to factors. We can use the function `sapply()` to see the column variable classes

In [ ]:
data_filtered<-data_filtered %>% 
mutate(`Sample type`= factor(`Sample type`),
      Trial = factor(Trial),
      Treatment = factor(Treatment),
      Species = factor(Species),
      `Days after inoculation`=factor(`Days after inoculation`))

sapply(data_filtered,class)

<br><br><br>
<div class="startTASK"></div>

In fox_data, create a new column called 'Consumption' which will contain values of avfood divided by weight
i.e Consumption =avfood/weight

<div class="endTASK"></div>
<br><br><br>

<div class="Q"><br><br>

What is the mean Consumption?
    
ANSWER:0.1544922 
    
<br><br><div class="qhelp"></div></div>

_______________________________________________________________________________________________

# group_by() and summarize() functions

Many data analysis tasks can be approached using the split-apply-combine paradigm: 

1. split the data into groups 
1. apply some analysis to each group
1. combine the results. 

dplyr makes this very easy through the use of the `group_by()` function. `group_by()` is often used together with `summarize()`, which collapses each group into a single-row summary of that group. `group_by()` takes as arguments the column names that contain the categorical variables for which you want to calculate the summary statistics. 

<img src="groupbysummarize.png" width="500px">

In the example below, we compute the mean and standard deviation of Photosynthesis measurements by Treatment.

In [ ]:
data_filtered %>% 
group_by(Treatment) %>% 
summarize(average=mean(Photosynthesis),
        dev=sd(Photosynthesis))

<div class="Q"><br><br>

What would the code inside the summarise function look like if we wanted to get the mean of Conductance?
    
ANSWER to be removed in final iteration: summarize(average=mean(Conductance)) <br><br>
    
<br><br><div class="qhelp"></div></div>

Let's take a look at our data graphically. We notice that the solid horizontal lines representing our average Wet and Drought values don't line up with the boxplot median. This indicates that our data might not be normally distributed. We may want to use the `median()` function to find the midpoint of the data instead. 

In [ ]:
data_filtered %>% 
group_by(Treatment) %>% 
ggplot()+
geom_boxplot(aes(x=Treatment, y=Photosynthesis, fill=Treatment))+
geom_hline(aes(yintercept=1.232575), color="red")+
annotate("text", x =0.7, y = 3,label ="Drought average = 1.23", size = 3,fontface =2, color="red")+
geom_hline(aes(yintercept=5.918684), color="blue")+
annotate("text", x =1.4, y = 7,label ="Wet average = 5.92", size = 3,fontface =2, color="blue")+
theme_classic()+
guides(fill=FALSE)

We can also group by mutliple columns.

In [ ]:
data_filtered %>% 
group_by(Treatment, Species) %>% 
summarize(median=median(Photosynthesis))

In [ ]:
data_filtered %>% 
group_by(Treatment, Species) %>%  
ggplot(aes(x=Species, y=Photosynthesis))+
geom_boxplot(aes(fill=Species))+
labs(x="")+
theme_classic()+
theme(axis.text.x=element_text(size=8),
     legend.position="bottom")+ 
facet_wrap(~Treatment) # separates plots by Treatment

<div class="Q"><br><br>

What would the code inside the group_by function look like if we wanted to group by both Date, Treatment and Species?
    
ANSWER to be removed in final iteration: group_by(Date, Treatment, Species) <br><br>
    
<br><br><div class="qhelp"></div></div>

We can also arrange the values in order of mean_ppb using the `arrange()` function. To sort in descending order, we need to add the `desc()` function.

In [ ]:
data_filtered %>% 
group_by(Treatment, Species) %>% 
summarize(median=median(Photosynthesis)) %>%
  arrange(desc(median))

Here is a table of some useful functions you can use with `summarize`. 

<img src="datasummaryfunction.png" width="500px">


<div class="Q"><br><br>

Using the help section, what would your arrange function code look like to arrange median in ascending order?
    
ANSWER to be removed in final iteration: arrange(median) <br><br>

multiple choice options: arrange(asc(median))

arrange(sort(median))

arrange(order(median))
    
<br><br><div class="qhelp"></div></div>

<br><br><br>
<div class="startTASK"></div>

In `fox_data`, find the mean weight by sex of the fox population.


<div class="endTASK"></div>
<br><br><br>

<div class="Q"><br><br>

What is the mean weight of Male foxes?
    
ANSWER to be removed in final iteration: <br><br>


<br><br><div class="qhelp"></div></div>

_______________________________________________________________________________________________

# Counting observations

When working with data, we often want to know the number of observations found for each factor or combination of factors. For this task, dplyr provides `count()`. Let's summarise the number of measurements made for each plant on each day.

In [ ]:
data_filtered %>%
    count(`Sample type`, `Days after inoculation`) %>% 
head(4)

We can also use `summarise()`,`n()` with `group_by()` to attain similar results.

In [ ]:
data_filtered %>%
    group_by(`Sample type`, `Days after inoculation`) %>%
    summarise(count = n())  %>% 
head(4)

There should be at least 10 measurements per plant per day (count >= 10). Say we wanted to see how many plants were there per Treatment and Species, we could use the function `n_distinct()` to count unique instances and remove duplicates. 

In [ ]:
plant.count<- data_filtered %>% 
group_by(Treatment,Species,`Days after inoculation`) %>% 
summarize(count=n_distinct(`Sample type`)) 

plant.count %>% head(4)

We can proceed with `arrange()` to sort the table according to a number of criteria so that we have a better comparison. For instance, we might want to arrange the table above in:

1. by Treatment and,
1. in descending order of the count:

In [ ]:
plant.count %>%
  arrange(Treatment, desc(count)) %>% 
head(10)

Let's see how may samples were taken for each species and treatment for each value of `Days after inoculation`. 

In [ ]:
plant.count %>% 
ggplot(aes(x=`Days after inoculation`, y=count, fill =Treatment))+
geom_col(position="dodge", color="black")+
theme_classic()+
facet_wrap(~Species)+
scale_y_continuous(expand=c(0,0))+ # removes space between plot and axis
theme(legend.position="bottom")

<div class="Q"><br><br>

What is the difference between the functions n() and n_distinct()?
    
ANSWER to be removed in final iteration: n() counts all observations. n_distinct() counts unique observations and excludes duplicates <br><br>
    
<br><br><div class="qhelp"></div></div>

# Preliminary data exmaination

Now that we've got the basics down, let's play around with our data. 

## Mean per plant

Let's take a look at the mean photosynthesis value per plant while keeping our other variables of interest. Here we are calculating a plant average for each plant. Note that we did not use the same plants every time we took measurements (1DroughtControl1 was only measured on days 9 and 22). We could only measure <30 plants per day, so we randomly selected them from the treatments each time we went to measure.

In [ ]:
Plant.Mean.Photo <- data_filtered %>% 
group_by(`Sample type`,Trial,Treatment,Species,`Days after inoculation`) %>% 
summarize(photo_average=mean(Photosynthesis,na.rm=TRUE))

Plant.Mean.Photo  %>% 
head(4)

In [ ]:
Plant.Mean.Photo %>% 
ggplot(aes(Species,photo_average,shape=Treatment,color=Species))+
geom_point(position="jitter")+
theme_classic()

Each plant is a sampling unit, so we want to conduct our analysis with a single value for each plant. If we include each measurement as a seperate observation, we are incorporating pseudoreplication because each measurement was not independant (10 measurements were made on a single plant–therefore, each measurement is dependant on the plant).

## Mean per group

Ok now that we have the means per plant, we can look at the means per trial, treatment, etc.

In [ ]:
Group.Mean.Photo<-Plant.Mean.Photo %>% 
group_by(Treatment,Species,`Days after inoculation`) %>% 
summarize(count=n(), 
          ave=mean(photo_average),
          dev=sd(photo_average), #standard deviation
          se=dev/sqrt(count) #standard error
         )

Group.Mean.Photo  %>% 
head(4)

<span class="note"></span> If there was only one plant in a group, the standard deviation, `dev` could not be calculated because there is no variation Some of the data might be missing and we would need to address this in real analysis.

We can plot the mean values from each Species and each Treatment on each day they were measured.

In [ ]:
ggplot(Group.Mean.Photo,aes(x=`Days after inoculation`,y=ave,color=Species))+
geom_point()+
facet_wrap(~Treatment,ncol=1) +
theme_classic()

This plot reveals some concerns: only control plants were measured on day 3 and no control plants were measured on day 5. Only one group of plants measured on day 36. Lots of errors in data, demonstrating the importance of exploring your data graphically!

Let's see if we can add estimates of error to our measurments.

In [ ]:
ggplot(Group.Mean.Photo,aes(x=`Days after inoculation`,y=ave,color=Species))+
geom_point()+
geom_errorbar(aes(ymin=ave-se, ymax=ave+se))+
facet_wrap(~Treatment,ncol=1) +
theme_classic()

_________________________________________________________________________

# Exporting data

Now that you have learned how to use dplyr to extract information from or summarize your raw data, you may want to export these new data sets to share them with your collaborators or for archival.

Similar to the `read_csv()` function used for reading CSV files into R, there is a `write_csv()` function that generates CSV files from data frames.

Before using `write_csv()`, we are going to create a new folder, data, in our working directory that will store this generated dataset. We don't want to write generated datasets in the same directory as our raw data. It's good practice to keep them separate. The folder for raw data should only contain the raw, unaltered data, and should be left alone to make sure we don't delete or modify it. In contrast, our script will generate the contents of the data directory, so even if the files it contains are deleted, we can always re-generate them.

Let's create a new folder/directory called `data_cleaned` using the `dir.create` function.

In [ ]:
dir.create(file.path("data_cleaned"))

Now that our data set and data_cleaned folder is ready, we can save it as a CSV file in our data folder.

In [ ]:
write_csv(combined_data,"./data_cleaned/combined_data.csv")

<div class="big_title">This is the end of lab</div>

*******************
*******************

Code below is for formatting of this lab. Do not alter!

In [2]:
cssFile <- '../css/custom.css'
IRdisplay::display_html(readChar(cssFile, file.info(cssFile)$size))

IRdisplay::display_html("<style>.Q::before {counter-increment: question_num;
    content: 'QUESTION ' counter(question_num) ': '; white-space: pre; }.T::before {counter-increment: task_num;
    content: 'Task ' counter(task_num) ': ';</style>")